# MLP - Multilayer Perceptron

Importing libraries

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from ipynb.fs.defs.Additional_metrics import *
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import itertools 
import time

## Trained on 30% samples of the original training dataset

In [2]:
train_data = pd.read_csv("datasets/fashion-mnist_train.csv")
X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]

test_data = pd.read_csv("datasets/fashion-mnist_test.csv")
X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]

X_train_std = (X_train - X_train.mean())/X_train.std()
X_test_std = (X_test - X_test.mean())/X_test.std()

In [3]:
X_train_30, X_test_30, y_train_30, y_test_30 = train_test_split(X_train_std, y_train, test_size=0.7, stratify=y_train, random_state=42)

"lbfgs" isnt good on big databases

In [4]:
x = [128,96,64,32,16] 
hd_list = list(itertools.combinations(x,3)) + list(itertools.combinations(x,2)) + [(128,128,128), (96,96,96), (64,64,64), (32,32,32), (128,128), (96,96), (64,64)]
print(hd_list)

[(128, 96, 64), (128, 96, 32), (128, 96, 16), (128, 64, 32), (128, 64, 16), (128, 32, 16), (96, 64, 32), (96, 64, 16), (96, 32, 16), (64, 32, 16), (128, 96), (128, 64), (128, 32), (128, 16), (96, 64), (96, 32), (96, 16), (64, 32), (64, 16), (32, 16), (128, 128, 128), (96, 96, 96), (64, 64, 64), (32, 32, 32), (128, 128), (96, 96), (64, 64)]


In [14]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc = []
i = 0

for hd in hd_list:
    for a in ["logistic", "tanh", "relu"]:
        for s in ["sgd", "adam"]:
            for bs in [50]:

                indexes = kf.split(X_train_30, y_train_30)
                acc_tmp = []
                time_tmp = []
                fin_conf_mat = np.zeros((len(np.unique(y_train_30)),len(np.unique(y_train_30))))

                for train_index, test_index in indexes:

                    X_trainCV1 = X_train_30.iloc[train_index,:]
                    y_trainCV1 = y_train_30.iloc[train_index]

                    X_testCV1 = X_train_30.iloc[test_index,:]
                    y_testCV1 = y_train_30.iloc[test_index]

                    start = time.time()

                    classifier1 = MLPClassifier(hidden_layer_sizes = hd, activation = a,
                                                    solver = s, batch_size = bs, learning_rate = "constant", 
                                                    learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                                                    random_state = 42, early_stopping = True, n_iter_no_change = 10,
                                                    validation_fraction = 0.1, verbose = False)
                    classifier1.fit(X_trainCV1.values, y_trainCV1.values)
                    y_pred1 = classifier1.predict(X_testCV1.values)
                    c1 = confusion_matrix(y_testCV1, y_pred1)               

                    acc_tmp.append(np.trace(c1)/sum(sum(c1)))
                    fin_conf_mat += c1
                    end = time.time()            

                    time_tmp.append(end-start)

                print(i,'. hd = ', hd,', a = ', a,', s = ', s,' bs = ', bs,', Average accuracy: ', round(np.mean(acc_tmp),8),' | Average execution time: ', round(np.mean(time_tmp),8), sep='')
                acc.append(np.mean(acc_tmp))
                i += 1
            
    print('')
      
print('-------------------')
print('Best accuracy is in iteration number: ', np.argmax(acc))

C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


0. hd = (128, 96, 64), a = logistic, s = sgd bs = 50, Average accuracy: 0.70033333 | Average execution time: 35.51353035
1. hd = (128, 96, 64), a = logistic, s = adam bs = 50, Average accuracy: 0.85816667 | Average execution time: 11.68062644
2. hd = (128, 96, 64), a = tanh, s = sgd bs = 50, Average accuracy: 0.85655556 | Average execution time: 17.03529773
3. hd = (128, 96, 64), a = tanh, s = adam bs = 50, Average accuracy: 0.85927778 | Average execution time: 11.87460213
4. hd = (128, 96, 64), a = relu, s = sgd bs = 50, Average accuracy: 0.86027778 | Average execution time: 14.57465854
5. hd = (128, 96, 64), a = relu, s = adam bs = 50, Average accuracy: 0.86177778 | Average execution time: 11.1457386



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


6. hd = (128, 96, 32), a = logistic, s = sgd bs = 50, Average accuracy: 0.43533333 | Average execution time: 15.43143797
7. hd = (128, 96, 32), a = logistic, s = adam bs = 50, Average accuracy: 0.85627778 | Average execution time: 12.49736838
8. hd = (128, 96, 32), a = tanh, s = sgd bs = 50, Average accuracy: 0.85622222 | Average execution time: 18.70481572
9. hd = (128, 96, 32), a = tanh, s = adam bs = 50, Average accuracy: 0.85738889 | Average execution time: 15.51651497
10. hd = (128, 96, 32), a = relu, s = sgd bs = 50, Average accuracy: 0.85911111 | Average execution time: 18.60212226
11. hd = (128, 96, 32), a = relu, s = adam bs = 50, Average accuracy: 0.86127778 | Average execution time: 12.69955301



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


...

144. hd = (128, 128), a = logistic, s = sgd bs = 50, Average accuracy: 0.83622222 | Average execution time: 35.97032299
145. hd = (128, 128), a = logistic, s = adam bs = 50, Average accuracy: 0.86433333 | Average execution time: 11.29567595
146. hd = (128, 128), a = tanh, s = sgd bs = 50, Average accuracy: 0.85866667 | Average execution time: 18.60812788
147. hd = (128, 128), a = tanh, s = adam bs = 50, Average accuracy: 0.86155556 | Average execution time: 11.34732251
148. hd = (128, 128), a = relu, s = sgd bs = 50, Average accuracy: 0.86066667 | Average execution time: 15.79837184
149. hd = (128, 128), a = relu, s = adam bs = 50, Average accuracy: 0.86777778 | Average execution time: 12.95058131



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


150. hd = (96, 96), a = logistic, s = sgd bs = 50, Average accuracy: 0.83733333 | Average execution time: 28.69170089
151. hd = (96, 96), a = logistic, s = adam bs = 50, Average accuracy: 0.86144444 | Average execution time: 10.45250831
152. hd = (96, 96), a = tanh, s = sgd bs = 50, Average accuracy: 0.85905556 | Average execution time: 11.61796803
153. hd = (96, 96), a = tanh, s = adam bs = 50, Average accuracy: 0.86261111 | Average execution time: 7.61993151
154. hd = (96, 96), a = relu, s = sgd bs = 50, Average accuracy: 0.86461111 | Average execution time: 11.05225406
155. hd = (96, 96), a = relu, s = adam bs = 50, Average accuracy: 0.86211111 | Average execution time: 8.05372581



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


156. hd = (64, 64), a = logistic, s = sgd bs = 50, Average accuracy: 0.83611111 | Average execution time: 24.08310857
157. hd = (64, 64), a = logistic, s = adam bs = 50, Average accuracy: 0.85888889 | Average execution time: 8.71372671
158. hd = (64, 64), a = tanh, s = sgd bs = 50, Average accuracy: 0.857 | Average execution time: 12.23152695
159. hd = (64, 64), a = tanh, s = adam bs = 50, Average accuracy: 0.85744444 | Average execution time: 6.49610925
160. hd = (64, 64), a = relu, s = sgd bs = 50, Average accuracy: 0.85927778 | Average execution time: 9.26402717
161. hd = (64, 64), a = relu, s = adam bs = 50, Average accuracy: 0.86188889 | Average execution time: 6.14458947

-------------------
Best accuracy is in iteration number:  149


#### Training on 30% training samples, test on test dataset

In [11]:
start = time.time()

classifier1 = MLPClassifier(hidden_layer_sizes = (128,64,16), activation = "relu",
                            solver = "adam", batch_size = 50, learning_rate = "constant", 
                            learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                            random_state = 42, early_stopping = True, n_iter_no_change = 10,
                            validation_fraction = 0.1, verbose = False)
classifier1.fit(X_train_30.values, y_train_30.values)
y_pred2 = classifier1.predict(X_test_std.values)
c2 = confusion_matrix(y_test, y_pred2, labels=[0,1,2,3,4,5,6,7,8,9])       

end = time.time()
print('Accuracy without reduction:' , np.trace(c2)/sum(sum(c2)), '| Average execution time: ', end - start)

Accuracy without reduction: 0.8738 | Average execution time:  13.816581964492798


.

## Trained on 100% samples of the original training dataset

In [12]:
train_data = pd.read_csv("datasets/fashion-mnist_train.csv")
X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]

test_data = pd.read_csv("datasets/fashion-mnist_test.csv")
X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]

X_train_std = (X_train - X_train.mean())/X_train.std()
X_test_std = (X_test - X_test.mean())/X_test.std()

In [13]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc = []

indexes = kf.split(X_train, y_train)
acc_tmp = []
time_tmp = []
fin_conf_mat = np.zeros((len(np.unique(y_train)),len(np.unique(y_train))))
        
for train_index, test_index in indexes:
    
    X_trainCV2 = X_train_std.iloc[train_index,:]
    y_trainCV2 = y_train.iloc[train_index]
    
    X_testCV2 = X_train_std.iloc[test_index,:]
    y_testCV2 = y_train.iloc[test_index]
    
    start = time.time()
                
    classifier2 = MLPClassifier(hidden_layer_sizes = (128,64,16), activation = "relu",
                            solver = "adam", batch_size = 50, learning_rate = "constant", 
                            learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                            random_state = 42, early_stopping = True, n_iter_no_change = 10,
                            validation_fraction = 0.1, verbose = False)
    classifier2.fit(X_trainCV2.values, y_trainCV2.values)
    y_pred3 = classifier2.predict(X_testCV2.values)
    c3 = confusion_matrix(y_testCV2, y_pred3)

    acc_tmp.append(np.trace(c3)/sum(sum(c3)))
    fin_conf_mat += c3
    end = time.time()

    
    time_tmp.append(end-start)
    print("        Accuracy: ", np.trace(c3)/sum(sum(c3)),'| Execution time:',end - start)
    
print("")    
print('-------------------')
print('Average accuracy:', np.mean(acc_tmp),'| Average execution time:',np.mean(time_tmp))   

        Accuracy:  0.8901666666666667 | Execution time: 50.61208724975586
        Accuracy:  0.8804166666666666 | Execution time: 27.89640474319458
        Accuracy:  0.8838333333333334 | Execution time: 35.0669322013855
        Accuracy:  0.8879166666666667 | Execution time: 28.194674015045166
        Accuracy:  0.8863333333333333 | Execution time: 29.48184561729431

-------------------
Average accuracy: 0.8857333333333333 | Average execution time: 34.25038876533508


##### Training on 100% training samples, test on test dataset

In [14]:
start = time.time()

classifier2 = MLPClassifier(hidden_layer_sizes = (128,64,16), activation = "relu",
                            solver = "adam", batch_size = 50, learning_rate = "constant", 
                            learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                            random_state = 42, early_stopping = True, n_iter_no_change = 10,
                            validation_fraction = 0.1, verbose = False)
classifier2.fit(X_train_std.values, y_train.values)
y_pred4 = classifier2.predict(X_test_std.values)
c4 = confusion_matrix(y_test, y_pred4, labels=[0,1,2,3,4,5,6,7,8,9])  

end = time.time()
print('Accuracy without reduction:' , np.trace(c4)/sum(sum(c4)), '| Average execution time: ', end - start)

Accuracy without reduction: 0.8885 | Average execution time:  38.890414237976074


.

## Trained on 100% samples with PCA reduction

In [20]:
train_data = pd.read_csv("datasets/fashion-mnist_train.csv")
X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]

test_data = pd.read_csv("datasets/fashion-mnist_test.csv")
X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]

X_train_std = (X_train - X_train.mean())/X_train.std()
X_test_std = (X_test - X_test.mean())/X_test.std()

In [21]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc = []
i = 0


for hd in hd_list:
    for a in ["logistic", "tanh", "relu"]:
        for s in ["sgd", "adam"]:

            indexes = kf.split(X_train, y_train)
            acc_tmp = []
            time_tmp = []
            fin_conf_mat = np.zeros((len(np.unique(y_train)),len(np.unique(y_train))))

            for train_index, test_index in indexes:

                X_trainCV3 = X_train_std.iloc[train_index,:]
                y_trainCV3 = y_train.iloc[train_index]

                X_testCV3 = X_train_std.iloc[test_index,:]
                y_testCV3 = y_train.iloc[test_index]

                pca = PCA(n_components=0.9)
                pca.fit(X_trainCV3)
                X_train_r = pca.transform(X_trainCV3)
                X_test_r = pca.transform(X_testCV3)

                start = time.time()
   
                classifier3 = MLPClassifier(hidden_layer_sizes = hd, activation = a,
                                    solver = s, batch_size = 50, learning_rate = "constant", 
                                    learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                                    random_state = 42, early_stopping = True, n_iter_no_change = 10,
                                    validation_fraction = 0.1, verbose = False)
                classifier3.fit(X_train_r, y_trainCV3)
                y_pred5 = classifier3.predict(X_test_r)
                c5 = confusion_matrix(y_testCV3, y_pred5, labels=[0,1,2,3,4,5,6,7,8,9])

    
                acc_tmp.append(np.trace(c5)/sum(sum(c5)))
                fin_conf_mat += c5
                end = time.time()

                time_tmp.append(end-start)
            print(i,'. hd = ', hd,', a = ', a,', s = ', s,', Average accuracy: ', round(np.mean(acc_tmp),8),' | Average execution time: ', round(np.mean(time_tmp),8), sep='')
            acc.append(np.mean(acc_tmp))
            i += 1
            
    print('')
      
print('-------------------')
print('Best accuracy is in iteration number: ', np.argmax(acc))  

C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural

0. hd = (128, 96, 64), a = logistic, s = sgd, Average accuracy: 0.87166667 | Average execution time: 79.15262098
1. hd = (128, 96, 64), a = logistic, s = adam, Average accuracy: 0.88413333 | Average execution time: 21.33763022
2. hd = (128, 96, 64), a = tanh, s = sgd, Average accuracy: 0.88141667 | Average execution time: 45.22950287
3. hd = (128, 96, 64), a = tanh, s = adam, Average accuracy: 0.8822 | Average execution time: 18.60253959
4. hd = (128, 96, 64), a = relu, s = sgd, Average accuracy: 0.88515 | Average execution time: 34.34107113
5. hd = (128, 96, 64), a = relu, s = adam, Average accuracy: 0.88665 | Average execution time: 17.37822509



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural

6. hd = (128, 96, 32), a = logistic, s = sgd, Average accuracy: 0.87083333 | Average execution time: 74.52746558
7. hd = (128, 96, 32), a = logistic, s = adam, Average accuracy: 0.88403333 | Average execution time: 20.56993108
8. hd = (128, 96, 32), a = tanh, s = sgd, Average accuracy: 0.88091667 | Average execution time: 42.18261147
9. hd = (128, 96, 32), a = tanh, s = adam, Average accuracy: 0.88131667 | Average execution time: 16.41915183
10. hd = (128, 96, 32), a = relu, s = sgd, Average accuracy: 0.88408333 | Average execution time: 30.18448653
11. hd = (128, 96, 32), a = relu, s = adam, Average accuracy: 0.88645 | Average execution time: 15.61001463



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural

...

144. hd = (128, 128), a = logistic, s = sgd, Average accuracy: 0.87793333 | Average execution time: 68.58685598
145. hd = (128, 128), a = logistic, s = adam, Average accuracy: 0.88811667 | Average execution time: 20.05606341
146. hd = (128, 128), a = tanh, s = sgd, Average accuracy: 0.88378333 | Average execution time: 40.20461092
147. hd = (128, 128), a = tanh, s = adam, Average accuracy: 0.88246667 | Average execution time: 16.64395623
148. hd = (128, 128), a = relu, s = sgd, Average accuracy: 0.88868333 | Average execution time: 33.57237148
149. hd = (128, 128), a = relu, s = adam, Average accuracy: 0.88465 | Average execution time: 18.0156054



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural

150. hd = (96, 96), a = logistic, s = sgd, Average accuracy: 0.87646667 | Average execution time: 55.38243208
151. hd = (96, 96), a = logistic, s = adam, Average accuracy: 0.88345 | Average execution time: 17.48412127
152. hd = (96, 96), a = tanh, s = sgd, Average accuracy: 0.88233333 | Average execution time: 36.80891867
153. hd = (96, 96), a = tanh, s = adam, Average accuracy: 0.88245 | Average execution time: 13.96171393
154. hd = (96, 96), a = relu, s = sgd, Average accuracy: 0.88618333 | Average execution time: 29.35513134
155. hd = (96, 96), a = relu, s = adam, Average accuracy: 0.88485 | Average execution time: 12.85991068



C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Korisnik\anaconda3\lib\site-packages\sklearn\neural

156. hd = (64, 64), a = logistic, s = sgd, Average accuracy: 0.87485 | Average execution time: 46.34880567
157. hd = (64, 64), a = logistic, s = adam, Average accuracy: 0.88131667 | Average execution time: 15.27951384
158. hd = (64, 64), a = tanh, s = sgd, Average accuracy: 0.88008333 | Average execution time: 28.25112576
159. hd = (64, 64), a = tanh, s = adam, Average accuracy: 0.87853333 | Average execution time: 10.44691324
160. hd = (64, 64), a = relu, s = sgd, Average accuracy: 0.88396667 | Average execution time: 24.18602414
161. hd = (64, 64), a = relu, s = adam, Average accuracy: 0.88023333 | Average execution time: 13.27248654

-------------------
Best accuracy is in iteration number:  148


##### Training on 100% training samples with PCA reduction, test on test dataset

In [13]:
pca = PCA(n_components=0.9)
pca.fit(X_train_std)
X_train_r = pca.transform(X_train_std)
X_test_r = pca.transform(X_test_std)

start = time.time()

classifier3 = MLPClassifier(hidden_layer_sizes = (128,128), activation = "relu",
                                    solver = "adam", batch_size = 50, learning_rate = "constant", 
                                    learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                                    random_state = 42, early_stopping = True, n_iter_no_change = 10,
                                    validation_fraction = 0.1, verbose = False)
classifier3.fit(X_train_r, y_train)
y_pred6 = classifier3.predict(X_test_r)
c6=confusion_matrix(y_test, y_pred6, labels=[0,1,2,3,4,5,6,7,8,9])

end = time.time()
print('Accuracy without reduction:', np.trace(c6)/sum(sum(c6)), '| Execution time:', end - start)

Accuracy without reduction: 0.8903 | Execution time: 18.281433582305908


In [5]:
pca = PCA(n_components=0.9)
pca.fit(X_train_std)
X_train_r = pca.transform(X_train_std)
X_test_r = pca.transform(X_test_std)

start = time.time()

classifier3 = MLPClassifier(hidden_layer_sizes = (128,128), activation = "relu",
                                    solver = "sgd", batch_size = 50, learning_rate = "constant", 
                                    learning_rate_init = 0.001, max_iter = 100, shuffle = True,
                                    random_state = 42, early_stopping = True, n_iter_no_change = 10,
                                    validation_fraction = 0.1, verbose = False)
classifier3.fit(X_train_r, y_train)
y_pred6 = classifier3.predict(X_test_r)
c6=confusion_matrix(y_test, y_pred6, labels=[0,1,2,3,4,5,6,7,8,9])

end = time.time()
print('Accuracy without reduction:', np.trace(c6)/sum(sum(c6)), '| Execution time:', end - start)

Accuracy without reduction: 0.8921 | Execution time: 61.15469145774841


In [14]:
print(c6)

[[865   0  11  10   3   0 102   0   8   1]
 [  5 973   2  14   0   0   6   0   0   0]
 [ 20   0 764  11 124   1  77   0   3   0]
 [ 31   8   4 899  28   1  27   0   2   0]
 [  0   0  41  22 867   0  68   0   2   0]
 [  0   0   0   0   0 941   1  35   7  16]
 [136   1  49  13  54   0 736   0  11   0]
 [  0   0   0   0   0  20   0 944   0  36]
 [  1   0   6   3   2   6  12   2 967   1]
 [  0   0   0   0   0   9   0  44   0 947]]


In [15]:
print(accuracy_per_class(c6, y_test.unique()))

For class 0 accuracy is: 0.9672
For class 1 accuracy is: 0.9964
For class 2 accuracy is: 0.9651
For class 3 accuracy is: 0.9826
For class 8 accuracy is: 0.9656
For class 6 accuracy is: 0.9904
For class 5 accuracy is: 0.9443
For class 4 accuracy is: 0.9863
For class 7 accuracy is: 0.9934
For class 9 accuracy is: 0.9893
-------------
0.9780599999999999


In [16]:
print(sensitivity_per_class(c6, y_test.unique()))

For class 0 sensitivity is: 0.865
For class 1 sensitivity is: 0.973
For class 2 sensitivity is: 0.764
For class 3 sensitivity is: 0.899
For class 8 sensitivity is: 0.867
For class 6 sensitivity is: 0.941
For class 5 sensitivity is: 0.736
For class 4 sensitivity is: 0.944
For class 7 sensitivity is: 0.967
For class 9 sensitivity is: 0.947
-------------
0.8902999999999999


## Examples of wrong classification